In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lecture 3A - Apply & Map, Misc

# Table of Contents
* [Lecture 3A - Apply & Map, Misc](#Lecture-3A---Apply-&-Map,-Misc)
	* &nbsp;
		* [Content](#Content)
		* [Learning Outcomes](#Learning-Outcomes)
	* [1. Functions and Dataframes - Using *apply()* and *applymap()*](#1.-Functions-and-Dataframes---Using-*apply%28%29*-and-*applymap%28%29*)
		* [Functions along an axis](#Functions-along-an-axis)
		* [Functions applied element-wise](#Functions-applied-element-wise)
	* [Dummy Variables](#Dummy-Variables)
	* [2. Removing Duplicates](#2.-Removing-Duplicates)
	* [3. Transpose](#3.-Transpose)


---

### Content

1. Applying functions to dataframes
2. Removing duplicates
3. Re-shaping dataframes with transpose
4. Shift operations for time series

### Learning Outcomes

At the end of this lecture, you should be able to:

* apply functions to dataframes
* remove duplicate rows in dataframes
* transpose dataframes
* apply shift operations to dataframes for time series data


---

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pylab import rcParams

%matplotlib inline

In [3]:
# Set some Pandas options as you like
pd.set_option('max_columns', 30)
pd.set_option('max_rows', 30)

In [4]:
rcParams['figure.figsize'] = 15, 10
rcParams['font.size'] = 20

## 1. Functions and Dataframes - Using *apply()* and *applymap()* 

Built-in or user-defines functions can be applied along the entire axes of a dataframe.

To apply a function to an entire axis (or multiple axes) of a dataframe, we resort to the apply() method, which can take an optional axis argument to determine if the axis is vertical/column-wise (0) or horizontal/row-wise (1).

### Functions along an axis

In [5]:
df = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                'three' : pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

df = df[['one','two','three']]
df

,one,two,three
a,-1.332077,0.568361,NaN
b,-0.707354,-0.862052,-2.075656
c,0.536219,-2.780842,-1.670609
d,NaN,0.266049,0.039817


Below is an example of applying a built in sum function 

In [6]:
df.apply(np.sum, axis=0)

one     -1.503212
two     -2.808484
three   -3.706447
dtype: float64

**Exercise**: Apply the mean function to the above dataframe in a row-wise manner.

In [7]:
df.apply(np.mean, axis=1)

a   -0.381858
b   -1.215020
c   -1.305077
d    0.152933
dtype: float64

**Exercise**: Apply the sum function to columns 'one' and 'two' only in a row-wise manner, and assign the result to a new column in the dataframe called 'four'.

In [8]:
df[['one','two']]

,one,two
a,-1.332077,0.568361
b,-0.707354,-0.862052
c,0.536219,-2.780842
d,NaN,0.266049


In [73]:
df['four'] = df.loc[:,"one":"two"].apply(np.sum,axis=1)
df

,one,two,three,four
a,1.376706,0.026747,NaN,1.403453
b,-1.089953,-0.203209,0.336187,-1.293162
c,-0.510041,-0.121366,-0.596772,-0.631408
d,NaN,-0.228504,0.722007,-0.228504


**Exercise**: Replace the missing value in both columns with the row-wise mean value.

In [12]:
df.isnull() 
df.apply(np.mean, axis=1)
df['one'] = df["one"].fillna(df.apply(np.mean, axis=1))
df['three'] = df["three"].fillna(df.apply(np.mean, axis=1))
df

,one,two,three
a,-1.332077,0.568361,-0.381858
b,-0.707354,-0.862052,-2.075656
c,0.536219,-2.780842,-1.670609
d,0.152933,0.266049,0.039817


**Exercise**: Calculate the column-wise product for the first and third columns only.     

In [13]:
#  df[["one","three"]]
df["one"]*df["three"]

a    0.508665
b    1.468223
c   -0.895812
d    0.006089
dtype: float64

**Exercise**: Write a function which calculates the sum of a vector and then returns the square of the sum. Once you have done this, apply your function to the dataframe in a row-wise manner, whilst creating a new column 'five', to which you will add insert the result. 

In [22]:
def square_of_summed_vector(x):
    result =  x.apply(np.sum,axis = 1)
    result = result**2
    return result

In [23]:
square_of_summed_vector(df)

a     1.312342
b    13.286471
c    15.329039
d     0.210497
dtype: float64

### Functions applied element-wise

The apply() method produces some form of aggregate calculations on the axes of a dataframe.  applymap() on the other hand extends us the flexibility of applying functions which manipulate single elements in a dataframe.

Say we would like to define a function which returns 'pos' for a positive number and alternatively 'neg'

In [24]:
def pos_neg_to_string(x):
    if x >= 0:
        return 'pos'
    else: return 'neg'

We can apply this to our dataframe as follows:

In [25]:
df.applymap(pos_neg_to_string)

,one,two,three
a,neg,pos,neg
b,neg,neg,neg
c,pos,neg,neg
d,pos,pos,pos


Having the ability to apply element-wise operations on dataframes is extremely useful when it comes to dataset cleaning and transformations.

Let's take a look at a sample from a real-world dataset used for gathering results from a survey:

In [26]:
assig = pd.read_csv("../datasets/surveySample.csv")
assig.head()

,OCCUPATION_M,supermarket spend in a week
0,e. Self-employed,c. $200 to $300
1,l. Home Duties,d. $300 to $400
2,i. Retired,b. $100 to $200
3,i. Retired,b. $100 to $200
4,h. Machinery operator/driver,b. $100 to $200


In [27]:
assig.OCCUPATION_M.head(20)

0                 e. Self-employed
1                   l. Home Duties
2                       i. Retired
3                       i. Retired
4     h. Machinery operator/driver
5                   l. Home Duties
6                     n. No Answer
7                       k. Student
8                       i. Retired
9                       a. Manager
10                        c. Sales
11                   m. Unemployed
12                      a. Manager
13                      i. Retired
14                 b. Professional
15                   m. Unemployed
16                      i. Retired
17                      i. Retired
18                      i. Retired
19                 b. Professional
Name: OCCUPATION_M, dtype: object

Clearly the values in this column need to be cleaned up.

Let's first find out what all the unique values are in this dataset.

In [28]:
assig.OCCUPATION_M.unique()

array(['e. Self-employed', 'l. Home Duties', 'i. Retired',
       'h. Machinery operator/driver', 'n. No Answer', 'k. Student',
       'a. Manager', 'c. Sales', 'm. Unemployed', 'b. Professional',
       'g. Labourer', 'f. Technician/trade worker',
       'd. Clerical/administration', 'j. Community/personal'],
      dtype=object)

In [31]:
assig.OCCUPATION_M.nunique()

14

We can now write a function that removes the first 3 characters in each entry in order to tidy the values.

In [32]:
def remove_first_three_chars(x):
    return x.replace(x[:3], '')

In [33]:
assig[['OCCUPATION_M']].applymap(remove_first_three_chars)

,OCCUPATION_M
0,Self-employed
1,Home Duties
2,Retired
3,Retired
4,Machinery operator/driver
...,...
64995,No Answer
64996,Labourer
64997,Self-employed
64998,Professional


In order to make the change permanent, we need to assign the result to the dataframe:

In [34]:
assig['OCCUPATION_M'] = assig[['OCCUPATION_M']].applymap(remove_first_three_chars)

## Dummy Variables


A dummy variable is a numerical variable used in data analysis to represent subgroups of the sample in under study. 

In research design, a dummy variable is often used to distinguish different treatment groups. This is accomplished by taking distinct values from a column and creating new columns out of them which are populated with 0 or 1 in order to indicate whether or not the particular data point belongs to this. 

This is a frequent operation that can be easily in Python.

In [37]:
assig['OCCUPATION_M']

0                    Self-employed
1                      Home Duties
2                          Retired
3                          Retired
4        Machinery operator/driver
                   ...            
64995                    No Answer
64996                     Labourer
64997                Self-employed
64998                 Professional
64999                      Retired
Name: OCCUPATION_M, Length: 65000, dtype: object

In [35]:
assig['OCCUPATION_M'].str.get_dummies()  # str -> int

,Clerical/administration,Community/personal,Home Duties,Labourer,Machinery operator/driver,Manager,No Answer,Professional,Retired,Sales,Self-employed,Student,Technician/trade worker,Unemployed
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64995,0,0,0,0,0,0,1,0,0,0,0,0,0,0
64996,0,0,0,1,0,0,0,0,0,0,0,0,0,0
64997,0,0,0,0,0,0,0,0,0,0,1,0,0,0
64998,0,0,0,0,0,0,0,1,0,0,0,0,0,0


We can also specify if there are multiple values within some cells that should be treated as separate columns. In this example we will say that the forward slash indicates a distinct value for which we would like to generate a column for.

In [36]:
assig['OCCUPATION_M'].str.get_dummies('/')

,Clerical,Community,Home Duties,Labourer,Machinery operator,Manager,No Answer,Professional,Retired,Sales,Self-employed,Student,Technician,Unemployed,administration,driver,personal,trade worker
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64995,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
64996,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64997,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
64998,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


**Exercise:** From the assignment dataset, consider the column 'supermarket spend in a week'. The '\$' character can cause issues in some applications. We want to clean up this column in such a way that the first 3 characters are replaced as well as the '\$' character, and we also want to change entries with 'No Answer' to reflect that they are actually missing values so replace them with np.NaN. Write a function to do this and apply this function to this column.

Verify that your code works. 

## 2. Removing Duplicates

Duplicate rows may be naturally occurring in some datasets or they might arise from input errors. In many instances, like machine learning, these duplicate entries need to be removed from the datasets. 

Dataframes provide straightforward functionality to remove such records.

Here is an example:


In [38]:
df = pd.DataFrame({'c1': ['one'] * 3 + ['two'] * 4,
                  'c2': [1, 1, 2, 3, 3, 4, 4]})
df

,c1,c2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


`drop_duplicates` returns a DataFrame where the duplicated rows **across all columns** are dropped:

In [39]:
df.drop_duplicates()

,c1,c2
0,one,1
2,one,2
3,two,3
5,two,4


We can also pass a particular column we  would like the duplicates removed from. Let's first make a change to the dataframe:

In [38]:
df.loc[1, 'c1'] = 'five'
df

,c1,c2
0,one,1
1,five,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [39]:
df.drop_duplicates(['c2'])

,c1,c2
0,one,1
2,one,2
3,two,3
5,two,4


Notice that `drop_duplicates` by default keep the first observed value combination.

## 3. Transpose

Transposing is a special form of reshaping tabular data in such a way that the rows become columns and likewise the columns become rows.

In [40]:
df = pd.DataFrame({'one' : pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                'two' : pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                'three' : pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})

df = df[['one','two','three']]
df

,one,two,three
a,0.591943,-1.163465,NaN
b,0.987666,0.524608,0.447575
c,0.166474,-1.125274,0.953977
d,NaN,-1.817459,1.906220


Transpose of a dataframe can be accomplished using either the transpose() method call  or simple .T

In [41]:
df.T

,a,b,c,d
one,0.591943,0.987666,0.166474,NaN
two,-1.163465,0.524608,-1.125274,-1.817459
three,NaN,0.447575,0.953977,1.906220


Transpose operations are not permanent unless you re-assign the result back tothe original dataframe.

In [42]:
df

,one,two,three
a,0.591943,-1.163465,NaN
b,0.987666,0.524608,0.447575
c,0.166474,-1.125274,0.953977
d,NaN,-1.817459,1.906220


**Exercise:** Slice and select out a dataframe with rows 'c' and 'd' and columns 'one' and 'two', then execute a transpose.  

In [53]:
df.loc[]

,one,two,three
c,0.166474,-1.125274,0.953977
